In [327]:
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [328]:
dataset_file_path = "resources/dataset_2021-5-26-10-14.csv" # Caminho para o arquivo onde há o data_set
test_size = 0.2 # Tamanho percentual do dataset de teste
# Lista com as colunas que serão usadas no dataset
feature_columns = ["default_3months","dividas_vencidas_qtd", "dividas_vencidas_valor", "acao_judicial_valor","month"]
saved_model_dir = "saved_model" # Pasta onde serão salvos o modelo e o scaler
saved_model_file = "classifier.joblib" # arquivo onde será salvo o modelo
saved_transform_file = "scaler.joblib" # arquivo onde será salvo o scaler


In [329]:
# Monta o dataset para o modelo a partir do dataset dado
# de forma que a quantidade de labels sejam iguais:


dataset = pd.read_csv(dataset_file_path, sep="\t", encoding="utf-8")
default_1 = dataset[dataset["default"]==1]
default_0 = dataset[dataset["default"]==0].sample(n=default_1["default"].count())
model_dataset = pd.concat([default_0,default_1]).sample(n=2*default_1["default"].count())

In [330]:
# Separa o dataset em features e classes
label = model_dataset["default"]
features_dataset = model_dataset.filter(feature_columns)

#  Separa as features e as classes em datasets de teste e treino
features_train, features_test, label_train, label_test = train_test_split(features_dataset.values, label.values, 
                                                    shuffle = True, 
                                                    test_size=test_size)


In [332]:
# Reescala os dados com o standard scaler e salva a transformação
# para transformações futuras

scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)
dump(scaler, f"{saved_model_dir}/{saved_transform_file}")

['saved_model/scaler.joblib']

In [333]:
# Treina o modelo usando regressão logística e salva para predições futuras
classifier = LogisticRegression()
classifier.fit(features_train, label_train)
dump(classifier, f"{saved_model_dir}/{saved_transform_file}")

['saved_model/scaler.joblib']

In [336]:
# Faz as predições e calcula a matriz de confusão

y_pred = classifier.predict(features_test)
cm = confusion_matrix(label_test, y_pred)
print(cm)

[[2906 1027]
 [1881 2004]]


In [335]:
# Calcula as métricas: accuracy, precision, recall

true_positive = cm[0][0]
true_negative = cm[1][1]
false_positive = cm[0][1]
false_negative = cm[1][0]

print("accuracy = ",(true_positive+true_negative)/(true_positive+true_negative+false_negative+false_positive))
print("precision = ",(true_positive)/(true_positive+false_positive))
print("recall = ",(true_positive)/(true_positive+false_negative))

accuracy =  0.6249680225121514
precision =  0.7334170854271357
recall =  0.6093945720250522
